## Install PyTorch

Colab notebooks come with many packages, but PyTorch isn't one of them. Fortunately we can execute shell commands, and so it can be easily installed using pip.

In [0]:
!pip install torch

### Verify installation and runtime
Google is kind enough to provide all of us with some free GPU compute time.

In [0]:
import torch
assert torch.__version__ == '0.4.1'
assert torch.cuda.device_count() > 0, 'Runtime > Change runtime type > Hardware accelerator > Choose GPU'

# Simple operations

This section will cover the basics of working with PyTorch tensors, including:
- creating tensors and inspecting their properties
- various ways of performing mathematical operations on them
- instructions for using the documentation
- NumPy integration
- broadcasting
- representation
- GPU usage
- serialization

## Factory functions

There are many functions you can use to create new tensors. Some of the most useful include:
- `torch.asarray` - like `np.asarray`
- `torch.ones` - one-filled 
- `torch.zeros` - zero-filled 
- `torch.full` - constant-filled 
- `torch.randn` - random normal numbers
- `torch.empty` - uninitialized values (for fast creation of temporaries)

All of those functions take in a shape argument, an optional `dtype` specifying the data type of individual elements (defaults to `float32`), and a device where the tensor should be allocated (e.g. can be a GPU, defaults to `'cpu'`).


In [0]:
x = torch.as_tensor([2., 3., 4., 5.])
y = torch.full_like(x, 5, dtype=torch.int64)
print(x, x.shape, x.dtype)
print(y, y.shape, y.dtype)

## Simple operations

Tensors support all Python operators, and over 200 specialized mathematical operations. All of them are available both as methods and as functions in the torch package.

In [0]:
z = torch.arange(4, dtype=torch.float)
print(x + z, x * z, x.dot(z), x.lgamma(), x > z)
print(torch.add(x, z), torch.mul(x, z), torch.dot(x, z), torch.lgamma(x), torch.gt(x, z))
print('Tensor functions:')
print('\n'.join(n for n in dir(x) if not n.startswith('_') and not n.endswith('_')))

Resizing: If you want to resize/reshape tensor, you can use ``torch.reshape``:



In [0]:
q = torch.randn(4, 4)
w = q.reshape(16)
r = q.reshape(-1, 8)  # the size -1 is inferred from other dimensions
p = q[None, :, 0]  # all of NumPy indexing is also supported, None creates a new dimension of size 1
print(q.shape, w.shape, r.shape, p.shape)

If you have a one element tensor, use ``.item()`` to get the value as a
Python number



In [0]:
s = torch.randn(1)
print(s)
print(s.item())

## In-place modifications

Expressions like `x + z` always allocate a new tensor for the result, to avoid modifying the arguments. However, sometimes it is more convenient to modify a tensor value in-place without allocating any intermediates.

> NOTE: operations that mutate tensors given as their arguments always have names ending with an underscore.

In [0]:
xc = x.clone() # .clone() will ensure that modifying xc won't affect the value of x
print(xc)
xc.add_(2)
print(xc)

## Documentation pointers

Complete documentation listing all available operations can be found at [the PyTorch website](https://pytorch.org/docs).

#### Exercise

`torch.permute` is an operation that lets you rearrange the order tensor dimensions in an arbitrary way. Find it's documentation and fill in the cell below to satisfy the assert.

In [0]:
t = torch.randn((2, 3, 4, 5, 6, 7))
assert t.permute(...).shape == (4, 3, 7, 2, 6, 5)

## Broadcasting

PyTorch tensors support broadcasting, according to NumPy rules.

![broadcasting](https://jakevdp.github.io/PythonDataScienceHandbook/figures/02.05-broadcasting.png =500x)

>> Image taken from [here](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html).

#### Exercise

Read [the NumPy guide for broadcasting](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html#general-broadcasting-rules), work out the shape of `c * d` and fill in the assert below to make it pass.

In [0]:
a = torch.randn(2, 1, 4)
b = torch.randn(7, 2, 3, 1)
# NOTE: we align the dimensions to the right to figure out the result shape!
#      a:    2, 1, 4
#      b: 7, 2, 3, 1
# result: 7, 2, 3, 4
assert a.add(b).shape == (7, 2, 3, 4)

c = torch.randn(8, 1, 1, 4, 5, 1)
d = torch.randn(6, 1, 4, 1, 1)
assert c.add(d).shape == (...)

## NumPy interoperability

PyTorch tensors can be casted into NumPy arrays (and the other way around) with almost no cost. The relevant functions are `.numpy()` and `torch.from_numpy` (or `torch.as_tensor`).

In [0]:
x = torch.randn(4)
print(x)
print(x.numpy(), type(x.numpy()))
print(torch.from_numpy(x.numpy()))
print(torch.as_tensor(x.numpy()))

## Using GPUs

To use a GPU simply call `.cuda()`, `.to('cuda')`, or specify `device='cuda'` when creating a tensor. GPU tensors support exactly the same set of operations as CPU tensors, but they are much faster for larger arrays.

> NOTE: While GPUs can be an order of magnitude faster than CPUs, they really start to shine only once the workload is big enough. Don't be surprised if toy benchmarks show that they are slower for tiny arrays.

In [0]:
xc = x.cuda()
mc = torch.ones(4, 4, device='cuda')
print(x.device, xc.device)
print(xc)
result = torch.matmul(mc, xc) + 2
print(result)
print(result.cpu())

## Serialization

PyTorch provides its own serialization methods, mimicking the standard `pickle` module. It's not recommended to use `pickle` with PyTorch tensors, because `torch.*` functions are more efficient and make it possible to e.g. load GPU tensors onto the CPU (whereas `pickle` would fail).

In [0]:
torch.save(x, 'x.pt')
assert (torch.load('x.pt') == x).all()

# Automatic differentiation

`torch.autograd` enables efficient differentiation of any functions operating on PyTorch tensors.

AD significantly increases memory usage, and because of that it is disabled by default. To enable it, specify `requires_grad=True` when creating tensors w.r.t. which you'll want to differentiate. Alternatively, if the formula for tensor instantiation is more complicated, use the `.requires_grad_()` method to set it.



In [0]:
x = torch.arange(4, dtype=torch.float, requires_grad=True)
print(x)

y = torch.randn(4).requires_grad_(True)
assert y.requires_grad
y.requires_grad_(False)
assert not y.requires_grad

Any result of an operation which had at least a single argument whith `requires_grad` will require it as well (you can think of it as of a viral flag).

In [0]:
z = x * y
print(z.requires_grad)

You can differentiate through more complicated functions, but remember that PyTorch can only compute gradients (Jacobians of scalar-valued functions), or Jacobian-vector products. It is possible to recover the full Jacobian, but you'll need to differentiate the function as many times as the number of its outputs.

In [0]:
def poly(x):
  return x ** 2 + 5 * x + 2

def d_poly(x):
  return 2 * x + 5

y = poly(x)
assert y.requires_grad
# torch.matmul(jacobian(poly), ones) == d_poly, because jacobian(poly) is a diagonal matrix
d_x, = torch.autograd.grad(y, x, torch.ones_like(y)) # d_x == d_y / d_x
assert (d_x == d_poly(x)).all()

A different way to obtain gradients is to use the `.backward()` method. This is the most common method and you'll find it in pretty much every single PyTorch script. Unlike `torch.autograd.grad`, this call has a side effect of accumulating the computed gradients in the `.grad` attributes of all inputs which required gradients.

In [0]:
x = torch.arange(4, dtype=torch.float, requires_grad=True)
# NOTE: We need .sum() to get a scalar that we can differentiate.
#       Since d x.sum() / d x == torch.ones_like(x), we get behavior equivalent to one above.
#       In any real-world examples this will be a scalar loss you'll be optimizing, so this won't
#       be necessary.
poly(x).sum().backward()
print(x.grad)
poly(x).sum().backward()
print(x.grad) # NOTE: this is 2x the initial value, because the gradients are accumulated.

## Locally disabling differentiation

Sometimes you might want to disable AD (e.g. to evaluate your model without optimizing it), and it would be very inconvenient to change `.requires_grad` of every single input or model parameter. Fortunately, there's `torch.no_grad()`, which is a context manager that completely turns of the AD logic, irrespectively of any flags.

In [0]:
x = torch.randn(4, requires_grad=True)
assert (x * 2).requires_grad
with torch.no_grad():
  assert x.requires_grad            # no-grad mode doesn't affect existing tensors,
  assert not (x * 2).requires_grad  # but any newly created ones won't affect AD

## Differentiating through control flow

One of the strenghts of `torch.autograd` is that it's completely unaffected by the actual code you write. Loops, conditionals, generators, coroutines - any Python construct can be differentiated successfully.

In [0]:
x = torch.randn(4, requires_grad=True)
y = 1 + torch.rand(()) # y will be a scalar value sampled uniformly from [1, 2]
print(y)
z = x
while z.norm() < 10:
  z = z * y
z.sum().backward()
print(x.grad)

## Exercise 1: Fit a linear model

Material which we've covered above is sufficient to start coding simple models. In this exercise you'll create a train a linear regression model. 

#### Model

Since both input and output have one dimension, our model can be expressed as

$$w \cdot x + b$$

where $w$ and $b$ are 1 dimensional tensors representing the weight and bias. Note that $x$ can have arbitrarily many elements (as long as it's 1D), and broadcasting will ensure that this expression is still correct (and will compute an output for every input element in parallel).

#### Loss

The loss function we'll use is called mean squared error and can be computed as:

$$\frac{1}{N}(\mathrm{output} - \mathrm{target})^2$$

where $N$ is the number of examples, $\mathrm{output}$ is the model output, and $\mathrm{target}$ is the expected output value.

#### Gradient descent

Our initial guess for the weight and bias will be probably far off, but we can use gradient descent to optimize those values. This amounts to taking the derivatives of the loss with respect to both the weight and bias, and performing an update, given as:

$$\theta' = \theta - \eta \frac{\delta L}{\delta \theta}$$

where $\eta$ is the learning rate, and $\frac{\delta L}{\delta \theta}$ is a gradient of the loss value.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

num_samples = 100
x = torch.linspace(-5, 5, steps=num_samples, dtype=torch.float)
y = x * 0.6 + torch.randn(num_samples) * 1.5

def model(x, W, b):
  return ... # Write out a formula for a linear model

lr = 1e-3
W = ... # Create a randomly initialized 1-element weight tensor
b = ... # Create a randomly initialized 1-element bias tensor

for i in range(20):
  pred = model(x, W, b)
  loss = ... # Write a formula for the MSE loss
  grad_W, grad_b = torch.autograd.grad(loss, (W, b))
  with torch.no_grad():
    W -= ... # Write out an gradient descent update for W
    ... # Write out an gradient descent update for b
  print(loss.item())

# Plot the data and the fitted model
plt.scatter(x, y)
plt.plot(x.numpy(), model(x, W, b).detach().numpy(), 'r')
plt.show()

## Exercise 2: Rewrite the code above using torch.nn

Our model can be easily represented by an `nn.Linear` layer from `torch.nn`. Let's rewrite the exercise from above using more standard methods.

> Do you know why did we have to reshape the dataset to be of size `(num_samples, 1)`? If not, check `nn.Linear` docs for shapes it expects.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

num_samples = 100
x = torch.linspace(-5, 5, steps=num_samples, dtype=torch.float).unsqueeze(1)
y = x * 0.6 + torch.randn(num_samples, 1) * 1.5

model = ...             # Use the docs for torch.nn to figure out how to create this model
optimizer = ...         # Use the docs for torch.optim package to create an SGD optimizer

for i in range(20):
  pred = model(x)
  loss = ...            # Use the MSE function from PyTorch
  optimizer.zero_grad() # Zero the .grad parameters (remember they accumulate!)
  loss.backward()       # Compute the gradients
  optimizer.step()      # Update parameters
  print(loss.item())

# Plot the data and the fitted model
plt.scatter(x, y)
plt.plot(x.numpy(), model(x).detach().numpy(), 'r')
plt.show()

## Exercise 3: Make the above model train on the GPU

Copy the code from above and modify it to train the model on the GPU. Of course it's an overkill to use it for models this small, but it will be important when we get to more complicated networks.

In [0]:
# Copy the code from above and modify it to train the model on the GPU

## Exercise 4: Fit a nonlinear model to a more complicated dataset

Linear models were enough to fit previous datasets. Can you build up a more complicated non-linear model using `torch.nn`? An example can be found e.g. [here](https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/feedforward_neural_network/main.py#L38-L49).

In [0]:
import numpy as np
import matplotlib.pyplot as plt

num_samples = 100
x = torch.linspace(-1, 4, steps=num_samples, dtype=torch.float).unsqueeze(1)
y = 0.5 * x ** 4 + -2.2 * x ** 3 + 1.22 * x ** 2 + -0.05 * x + torch.randn(num_samples, 1) * 1.5

model = ...             # Use the docs for torch.nn to figure out how to create this model
optimizer = ...         # Use the docs for torch.optim package to create an SGD optimizer

for i in range(2000):
  pred = model(x)
  loss = ...            # Use the MSE function from PyTorch
  optimizer.zero_grad() # Zero the .grad parameters (remember they accumulate!)
  loss.backward()       # Compute the gradients
  optimizer.step()      # Update parameters
  if i % 100 == 0:
    print(loss.item())

# Plot the data and the fitted model
plt.scatter(x, y)
plt.plot(x.numpy(), model(x).detach().numpy(), 'r')
plt.show()